In [ ]:
import base64
import cv2
import numpy as np
import sys
import coils
import redis
import pickle
import multiprocessing 

In [ ]:
cam_name = "cam_"
shape_name = "shape_"

In [ ]:
class Writer:

	def __init__(self):
		self.__store = redis.Redis()

	def writer(self, video_path, idx):
		
		cap = cv2.VideoCapture(video_path)
		
		# Monitor the framerate at 1s, 5s, 10s intervals.
		fps = coils.RateTicker((1, 5, 10))

		while True:
			ret, frame = cap.read()
			if not ret:
				break

			h, w, c = frame.shape

			# reshape or flatten the frame into a string
			reshaped_image = frame.reshape(1, -1)
			
			# base encode string 
			# encoded_image = base64.b64encode(res_img).decode("utf-8")
			encoded_image = base64.b64encode(reshaped_image.tobytes())
			
			# store the encoded image against corresponding video/camera name - key-value pairs
			self.__store.set(cam_name+idx, encoded_image)
			# store shape of each frame with corresponding camera/video name string
			self.__store.set(shape_name+idx, pickle.dumps((h, w, c)))
			
			cv2.imshow('Video To Redis', frame)

			k = cv2.waitKey(1)

			if k == 27:
				cv2.destroyAllWindows()
				cap.release()
				break

			text = '{:.2f}, {:.2f}, {:.2f} fps'.format(*fps.tick())

			print(text)

		cap.release()


	def run(self):
		video_list = ['/home/acer/workspace/videos/Eng_Store_vid.mp4', '/home/acer/workspace/videos/Hot_Mill_View_vid.mp4']
		processes = []

		for i, video_path in enumerate(video_list):
			p = multiprocessing.Process(target=self.writer, args=(video_path, str(i),))
			p.start()
			processes.append(p)

		for p in processes:
			p.join()


In [ ]:
if __name__ == '__main__':
    obj = Writer()
    obj.run()